In [1]:
import gc
from pathlib import Path

import torch 
import transformers4rec.torch as tr
from transformers4rec.torch import Trainer
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch.ranking_metric import NDCGAt, RecallAt
from transformers4rec.torch.utils.examples_utils import wipe_memory
from transformers4rec.torch.utils.data_utils import MerlinDataLoader

from merlin.io import Dataset
from merlin.schema import Schema

/usr/local/lib/python3.10/dist-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")


In [2]:
data_path = Path.cwd() / 'data'

## LB

In [3]:
lb_in = data_path / 'lb'
lb_out = data_path / 'lb/model'
if not lb_out.is_dir():
    lb_out.mkdir()

In [4]:
schema_path = lb_in / 'processed_nvt/part_0.parquet'

x_cat_names = ['aid-list', 'type-list']
x_cont_names = ['product_recency_day_log_norm-list', 't_dow_sin-list', 't_dow_cos-list']

train = Dataset(schema_path.as_posix())
schema = train.schema
schema = schema.select_by_name(x_cat_names + x_cont_names)

In [5]:
sequence_length = 20
d_model = 256

inputs = tr.TabularSequenceFeatures.from_schema(
    schema,
    max_sequence_length=sequence_length,
    aggregation='concat',
    masking='mlm',
)

trainsformer_config = tr.XLNetConfig.build(
    d_model=d_model, n_head=32, n_layer=3, total_sequence_length=sequence_length
)

body = tr.SequentialBlock(
    inputs, tr.MLPBlock([d_model]), tr.TransformerBlock(trainsformer_config, masking=inputs.masking)
)

head = tr.Head(
    body,
    tr.NextItemPredictionTask(
        weight_tying=True,
        softmax_temperature=1.57,
        metrics=[
            NDCGAt(top_ks=[20, 40], labels_onehot=True),
            RecallAt(top_ks=[20, 40], labels_onehot=True)
        ]
    )
)

model = tr.Model(head)

Projecting inputs of NextItemPredictionTask to'64' As weight tying requires the input dimension '256' to be equal to the item-id embedding dimension '64'


In [6]:
training_args = T4RecTrainingArguments(
    output_dir=lb_out.as_posix(),
    max_sequence_length=20,
    data_loader_engine='merlin',
    num_train_epochs=2, 
    dataloader_drop_last=False,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=0.0005,
    lr_scheduler_type='cosine',
    learning_rate_num_cosine_cycles_by_epoch=1.5,
    report_to=[],
    save_steps=1000,
    logging_steps=1000,
    save_total_limit=3,
    evaluation_strategy='steps',
    metric_for_best_model='/next-item/recall_at_40',
    load_best_model_at_end=True,
    predict_top_k=40
)

trainer = Trainer(
    model=model,
    args=training_args,
    schema=schema,
    compute_metrics=True,
)

In [7]:
## %%time

sessions_path = lb_in / 'sessions_by_day'
start_window_index = 1
end_window_index = 27

for time_index in range(start_window_index, end_window_index):
    time_index_train = time_index
    time_index_eval = time_index + 1
    
    train_path = (sessions_path / f'{time_index_train}/train.parquet').as_posix()
    eval_path = (sessions_path / f'{time_index_eval}/valid.parquet').as_posix()

    print('*'*20)
    print("Launch training for day %s are:" %time_index)
    print('*'*20 + '\n')

    trainer.train_dataset_or_path = train_path
    trainer.eval_dataset_or_path = eval_path
    
    trainer.reset_lr_scheduler()
    trainer.train()
    trainer.state.global_step +=1
    
    train_metrics = trainer.evaluate(metric_key_prefix='eval')
    print('*'*20)
    print("Eval results for day %s are:\t" %time_index_eval)
    print('\n' + '*'*20 + '\n')
    for key in sorted(train_metrics.keys()):
        print(" %s = %s" % (key, str(train_metrics[key]))) 
    wipe_memory()

********************
Launch training for day 1 are:
********************



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,/next-item/ndcg At 20,/next-item/ndcg At 40,/next-item/recall At 20,/next-item/recall At 40,/loss
1000,12.849200,No log,0.004422,0.005949,0.010922,0.018418,12.430210
2000,12.042800,No log,0.045361,0.049619,0.081654,0.102492,11.681057
3000,11.345600,No log,0.127400,0.132314,0.183440,0.207468,11.104817
4000,10.919000,No log,0.169366,0.173759,0.224691,0.246173,10.763364
5000,10.537100,No log,0.183229,0.187643,0.238646,0.260175,10.605979
6000,10.467800,No log,0.192131,0.196595,0.247163,0.268944,10.543098
7000,10.335300,No log,0.191907,0.196357,0.246959,0.268646,10.540532
8000,10.115500,No log,0.196310,0.200599,0.251501,0.272449,10.509752
9000,10.205900,No log,0.203240,0.207523,0.258918,0.279835,10.402840
10000,10.040400,No log,0.217434,0.221725,0.274822,0.295770,10.243583


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



********************
Eval results for day 2 are:	

********************

 eval_/loss = 8.63762092590332
 eval_/next-item/ndcg_at_20 = 0.28581759333610535
 eval_/next-item/ndcg_at_40 = 0.2933831214904785
 eval_/next-item/recall_at_20 = 0.3830028176307678
 eval_/next-item/recall_at_40 = 0.4199327528476715
 eval_runtime = 32.2132
 eval_samples_per_second = 1975.834
 eval_steps_per_second = 61.745
********************
Launch training for day 2 are:
********************



...

********************
Eval results for day 25 are:	

********************

 eval_/loss = 9.054166793823242
 eval_/next-item/ndcg_at_20 = 0.2494548112154007
 eval_/next-item/ndcg_at_40 = 0.2574363946914673
 eval_/next-item/recall_at_20 = 0.344379186630249
 eval_/next-item/recall_at_40 = 0.38338926434516907
 eval_runtime = 5.1193
 eval_samples_per_second = 1862.772
 eval_steps_per_second = 58.212
********************
Launch training for day 25 are:
********************



Step,Training Loss,Validation Loss


********************
Eval results for day 26 are:	

********************

 eval_/loss = 9.20311450958252
 eval_/next-item/ndcg_at_20 = 0.2506013512611389
 eval_/next-item/ndcg_at_40 = 0.2575730085372925
 eval_/next-item/recall_at_20 = 0.34202125668525696
 eval_/next-item/recall_at_40 = 0.37619680166244507
 eval_runtime = 4.0525
 eval_samples_per_second = 1855.657
 eval_steps_per_second = 57.989
********************
Launch training for day 26 are:
********************



Step,Training Loss,Validation Loss


********************
Eval results for day 27 are:	

********************

 eval_/loss = 9.085286140441895
 eval_/next-item/ndcg_at_20 = 0.24572838842868805
 eval_/next-item/ndcg_at_40 = 0.25360390543937683
 eval_/next-item/recall_at_20 = 0.3383556604385376
 eval_/next-item/recall_at_40 = 0.377046138048172
 eval_runtime = 2.9258
 eval_samples_per_second = 1837.437
 eval_steps_per_second = 57.42


In [8]:
trainer.save_model((lb_out / 't4r').as_posix())